In [1]:
import azure.cosmos
from azure.cosmos.partition_key import PartitionKey

In [2]:
#create the database
database = cosmos_client.create_database_if_not_exists('RetailIngest')


In [3]:
#create the container with default index(which indexes all properties in the documents)
container1 = database.create_container_if_not_exists(id='WebsiteMetrics', partition_key=PartitionKey(path='/CartID'))

In [ ]:
#insert data into the WebsiteMetrics
%%upload --databaseName RetailIngest --containerName WebsiteMetrics --url https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData.json

In [5]:
#create another container, excluding the index on the property you are quering 
#ex; your query is "select * from c where c.Price >=300" and Price is not indexed
container_definition ={
  # Specify your indexing policy here
    "indexingPolicy": {
    "indexingMode": "consistent",
    "automatic": True,
    "includedPaths": [
        {
            "path": "/CartID/*"
        }
    ],
    "excludedPaths": [
        {
            "path": "/*"
        },
        {
            "path": "/\"_etag\"/?"
        }
    ]
    }
 }
#create the container2 using above index definition
container2 = database.create_container_if_not_exists(id='WebsiteMetricsNoIndex', partition_key=PartitionKey(path='/CartID'),indexing_policy=container_definition["indexingPolicy"])


In [5]:
#insert data into the WebsiteMetricsNoIndex
%%upload --databaseName RetailIngest --containerName WebsiteMetricsNoIndex --url https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData.json

Documents successfully uploaded to WebsiteMetricsNoIndex3 
Total number of documents imported: 
  Success: 2654 
  Failure: 0 
Total time taken : 00:00:36 hours 
Total RUs consumed : 16693.660000001135

In [9]:
#execute the query which is using index
# expand "Hide Query Metrics" and observe the taken for the query
%%sql --database RetailIngest --container WebsiteMetrics --output df_cosmos
select * from c where c.Price >=300

¶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 2.02 queryCompileTimeInMs : 0.08 queryLogicalPlanBuildTimeInMs : 0.02 queryPhysicalPlanBuildTimeInMs : 0.12000000000000001 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 1.44 indexLookupTimeInMs : 0.12000000000000001 instructionCount : 442 documentLoadTimeInMs : 1.18 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 54 retrievedDocumentSize : 28112 outputDocumentCount : 54 outputDocumentSize : 28264 writeOutputTimeInMs : 0.04 averageIndexUtilizationRation : 1

In [10]:
#execute the query which is not using index
# expand "Hide Query Metrics" and observe the taken for the query
%%sql --database RetailIngest --container WebsiteMetricsNoIndex --output df_cosmos
select * from c where c.Price >=300

¶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 7.81 queryCompileTimeInMs : 0.05 queryLogicalPlanBuildTimeInMs : 0.01 queryPhysicalPlanBuildTimeInMs : 0.04 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 7.56 indexLookupTimeInMs : 0 instructionCount : 16037 documentLoadTimeInMs : 6.49 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 2654 retrievedDocumentSize : 1366721 outputDocumentCount : 54 outputDocumentSize : 28215 writeOutputTimeInMs : 0.02 averageIndexUtilizationRation : 0.02